# 基于Python的拖曳工况下后视镜视野仿真

## 基于Python的后视镜视野仿真方法
本节将详细介绍如何使用Python搭建一个拖车后视镜视野仿真工具。仿真分为几部分：几何建模（车辆、拖车、后视镜和驾驶员眼点位置）、视线投射计算（模拟光线从驾驶员眼睛通过后视镜反射看到拖车后方的过程），以及结果输出。

### 1. 仿真场景定义
首先，需要建立一个简化的3D场景模型。考虑到我们不依赖复杂的3D建模软件（如Blender）直接导入完整模型，我们采用参数化的简化模型描述车辆、后视镜和拖车：
+ **坐标系** ：建立右手坐标系，通常以牵引车（拖车头）为参考。本文采用汽车纵向为x轴（x轴向前，朝车辆行驶方向），横向为y轴（向左为正y），竖直为z轴（向上为正z）。为了方便计算，我们可将坐标系原点放在驾驶员眼睛位置下方地面位置。这样能简化后续视线计算。
+ **驾驶员眼睛和坐姿**：需要定义驾驶员眼点相对于车辆的参数，比如高度（眼高），座椅位置等。根据人体工程，一般驾驶员眼高约1.1~1.3米（具体可依据SAE J941眼椭圆确定不同分位的眼点范围）。为仿真不同身高和坐姿，我们可以将眼点作为可变参数。本示例假定某一固定眼点高度，如1.2米，位于车辆左座靠近中线的位置。之后在扩展场景中，我们会考虑不同眼点的变化。
+ **后视镜参数** ：定义后视镜的位置和朝向，以及镜面尺寸和类型。假设我们以驾驶员侧外后视镜为例，其位置可用相对于驾驶员眼的坐标表示，如在驾驶员左侧约0.5米、前方0.2米、稍低0.1米处（由于侧后视镜通常位于车窗前柱附近，略比眼睛低一点）。镜面的朝向用法向量表示，即镜面法线垂直于镜面平面、指向反射光线进入眼睛的方向。镜面尺寸则用宽度w和高度h表示（例如镜面宽度0.2m、高度0.15m）。镜面曲率：对于平面镜，曲率半径无限大；若为凸面镜，可给定曲率半径R（例如典型汽车凸面镜R约1200~1500mm）。在仿真中，不同曲率主要影响视野角度：平面镜的视场角等于镜子物理尺寸产生的几何遮挡角，而凸面镜等效扩大了可视角。我们可以通过调整镜面曲率参数，或等效地调整投射算法，实现对凸面镜的模拟（后面详述实现方法）。
+ **拖车模型**：拖车简化为规则长方体或多边形棱柱。主要参数是宽度（例如2.4m）、高度（例如2.5m）、以及相对于牵引车的位置（拖车前端与牵引车后部的距离，即挂接长度，假设比如车尾到拖车头距离1m）。为简单起见，我们假定拖车正后方在牵引车正后方，例如拖车后部相对于驾驶员眼睛原点在x轴负方向若干米处。举例来说，牵引车长4m，拖车长6m，则拖车尾部可能在驾驶员眼后方约10m处。我们将重点关注拖车尾部的四个边缘点（左上、左下、右上、右下角）以及地面上的一些参考点，用于判断这些点在后视镜中的可见性。

如下是设定的基本参数，使用Python字典定义参数(单位：m)。

In [1]:
import numpy as np
# 眼点
eye_pos = np.array([0.0, 0.0, 1.2])
# 镜面 
left_mirror = {
    "mid": np.array([0.6, 0.8, 1.1]), 
    "width": 0.2,  # 镜面宽度
    "height": 0.15,  # 镜面高度
    "curvature_radius": None,  # 曲率半径
}
right_mirror = {
    "mid": np.array([0.6, -1.8, 1.1]),  
    "width": 0.2,  # 镜面宽度
    "height": 0.15,  # 镜面高度
    "curvature_radius": None,  # 曲率半径
}
trailer = {
    "distance": 8.0,  # 拖车尾部相对驾驶员眼睛的距离（例如眼睛到拖车尾10m，这里简化为8m用于演示）
    "width": 2.4,  
    "height": 2.5,  
}

### 2. 后视镜朝向计算（镜面法线确定）

后视镜的朝向决定了驾驶员能通过镜子看到的区域。我们需要确定**镜面法线**（normal）方向。对于平面镜，理想情况下镜面应**平分**驾驶员眼睛到目标物之间的夹角。

具体而言，以左后视镜为例，如果希望后视镜能够将拖车后方的某一点反射到驾驶员眼中，镜面的法线**应当位于驾驶员视线方向和该目标方向的角平分面上**。例如，我们希望镜子正好瞄准拖车后部中部，那么可以让镜面法线指向眼睛和拖车后部中心连线的角平分方向。

**计算方法如下：**

- 设 $E$ 为眼睛位置，$M$ 为后视镜左上位置，$T$ 为我们希望镜子看到的目标点位置（例如拖车后部某关键点）。我们可以先计算向量 $\mathbf{ME} = E - M$（从镜子指向眼睛）和 $\mathbf{MT} = T - M$（从镜子指向目标）的单位方向向量。

- 镜面法线方向向量 $\mathbf{n}$ 估计就是上述两个单位向量的和，然后再归一化：

  $$
  \mathbf{n} = \frac{\frac{\mathbf{ME}}{|\mathbf{ME}|} + \frac{\mathbf{MT}}{|\mathbf{MT}|}}{\left|\frac{\mathbf{ME}}{|\mathbf{ME}|} + \frac{\mathbf{MT}}{|\mathbf{MT}|}\right|}
  $$

这实际上得到了位于 $\mathbf{ME}$ 和 $\mathbf{MT}$ 实角平分线方向的单位向量 $\mathbf{n}$。这种法线设定可以保证 $\mathbf{ME}$ 与 $\mathbf{MT}$ 的夹角关于法线的夹角相等，即满足反射在角度上的对称要求。

如下代码实现这一计算，选取拖车左后下角作为目标点，目标点坐标为 $T =(- trailer["distance"], trailer["width"]/2, 0)$。

In [2]:
E = eye_pos # 眼睛位置
M = left_mirror["mid"]  # 左镜面中心
T = np.array([-trailer["distance"], trailer["width"]/2, 0.0]) # 拖车左后下角位置

EM = E - M  # 眼睛到左镜面的向量
TM = T - M  # 左镜面到拖车左后下角的向量
u_EM = EM / np.linalg.norm(EM)  # 单位化向量
u_TM = TM / np.linalg.norm(TM)  # 单位化向量

# 计算法线向量
n_Mirror = u_EM + u_TM  # 法线向量"""  """
n_Mirror /= np.linalg.norm(n_Mirror)  # 单位化法线向量
print("Mirror normal =", n_Mirror)

Mirror normal = [-0.90411631 -0.42700494 -0.01550705]


In [39]:
# 自定义修改法线，如果不修改则拖车左下角必可见
n_Mirror = np.array([-0.8, -0.4, -0.1])  # 假设镜面法线垂直向上
n_Mirror /= np.linalg.norm(n_Mirror)  # 单位化法线向量
print("Mirror normal =", n_Mirror)

Mirror normal = [-0.88888889 -0.44444444 -0.11111111]


这表示镜面法线大致朝向车辆后方（x负方向），略向内（y负表示朝向驾驶员一侧）并略微向下（z负很小）。

### 3. 构建镜面位置

此时，我们已经得到了镜面平面的朝向（法线）。接下来需要建立镜面的平面坐标系，以便判断任意一点通过镜子反射后的成像位置。

- **镜面平面坐标系**：我们可以取镜面左上角 $M$ 为原点，定义两个单位向量分别作为镜面的水平方向 $\mathbf{u}_h$ 和垂直方向 $\mathbf{u}_v$。由于镜面大致是竖直的，我们可以这样来**近似**：令$\mathbf{u}_h$ z方向为0, 三个向量之间互相垂直。

- **构造水平向量 $\mathbf{u}_h$**: 我们希望 $\mathbf{u}_h$ z分量为 0，并且与 $\mathbf{n}_{Mirror}$ 正交。考虑
$$
\mathbf{h}' =
\begin{pmatrix}
-\,\hat n_y \\[4pt]
\;\hat n_x \\
0
\end{pmatrix}, 
\mathbf{h}
= \frac{\mathbf{h}'}{\|\mathbf{h}'\|}
$$
- **构造近似垂直向量 $\mathbf{u}_v$**: 利用右手坐标系下的叉乘生成与前两者都正交的向量。

In [3]:
u_h = np.array([n_Mirror[1], -n_Mirror[0], 0.0])  # 水平方向向量
u_h /= np.linalg.norm(u_h)  # 单位化水平方向向量
u_v = np.cross(n_Mirror, u_h)  # 垂直方向向量
u_v /= np.linalg.norm(u_v)  # 单位化垂直方向向
print("Mirror horizontal vector =", u_h)
print("Mirror vertical vector =", u_v)

Mirror horizontal vector = [-0.42705629  0.90422504  0.        ]
Mirror vertical vector = [ 0.01402186  0.00662238 -0.99987976]


### 4. 视线投射与可见性计算

有了镜面的定义后，我们可以模拟判断拖车哪些区域能够被驾驶员通过镜子看到。核心思想是**光线反射判断**：某一点是否能在镜中被看见，取决于从该点出发的“光线”经镜子反射是否能进入驾驶员眼中。

使用**虚像法**：将驾驶员的眼睛相对于镜面做仿照映射，得到眼睛的“虚像”位置；那么只要目标点与这个虚拟眼睛连线能够穿过镜面，且穿过的位置在镜面范围之内，则该点可见。

具体步骤如下：

#### 1. 计算驾驶员眼睛相对于镜面的镜像位置：

镜面已知法线 $\mathbf{n}_{Mirror}$，和通过镜面中心的平面。对眼睛点 $E$ 作平面对称，得到 $E'$。

- **点到M的向量，在法向量方向的距离**：
$$
d = (E - M) \cdot \mathbf{n}_{Mirror}
$$

$$
E' = E - 2 d \cdot \mathbf{n}_{Mirror}
$$

这里 $(E - M) \cdot \mathbf{n}_{Mirror}$ 是眼睛到镜面平面的法向距离，乘以 $\mathbf{n}_{Mirror}$ 得到该方向的偏移向量。

#### 2. 光线-平面交：

对于拖车上的任一点 $P$（可以是我们选取的各个边缘点等），考虑从 $P$ 出发指向虚拟眼睛 $E'$ 的线段。这条线可参数化为：

$$
L(t) = P + t \cdot (E' - P),\quad t \in \mathbb{R}
$$

在 $t=0$ 时位于 $P$，$t=1$ 时位于 $E'$。

求此线段与镜面（过 $M$，法向为 $\mathbf{n}$）的交点，即令 $L(t)$ 满足：

$$
(L(t) - M) \cdot \mathbf{n} = 0
$$

解此方程可以得到交点参数：

$$
t^* = \frac{\mathbf{n} \cdot (M - P)}{\mathbf{n} \cdot (E' - P)}
$$

>（注意分母为线方向向量在法线方向的分量，需 $\ne 0$，否则说明线与平面平行；对正确设置的虚像方法，分母不应为0）

计算得到 $t^*$ 后，求出交点 $X = P + t^* (E' - P)$，理论上 $X$ 应该落在镜面平面内。

#### 3. 判断交点是否在镜面范围内：

将交点 $X$ 转换到镜面局部坐标系，即计算向量 $\vec{MX} = X - M$，然后计算其在 $\mathbf{u}_h$ 与 $\mathbf{u}_v$ 方向上的坐标：

- $x_{\text{local}} = \vec{MX} \cdot \mathbf{u}_h$
- $y_{\text{local}} = \vec{MX} \cdot \mathbf{u}_v$

如果满足：

$$
0 \leq x_{\text{local}} \leq w,\quad 0 \leq y_{\text{local}} \leq h
$$

（其中 $w$, $h$ 为镜面宽高），则交点在镜面有效范围内，说明发出的反射光线可以从镜面反射进入驾驶员——**该点对驾驶员可见**。反之则不可见。


In [4]:
# 计算眼点E的镜像点
E_ref = E - 2 * np.dot(EM, n_Mirror) * n_Mirror

# 定义拖车后部四个角点的坐标
diff_d = abs(left_mirror["mid"][1]+right_mirror["mid"][1]) / 2
trailer_corners = {
    "left_bottom": np.array([-trailer["distance"], trailer["width"]/2-diff_d, 0]),  # 拖车左上角
    "left_top": np.array([-trailer["distance"], trailer["width"]/2-diff_d, trailer["height"]]),  # 拖车左下角
    "right_bottom": np.array([-trailer["distance"], -trailer["width"]/2-diff_d, 0]),  # 拖车右下角
    "right_top": np.array([-trailer["distance"], -trailer["width"]/2-diff_d, trailer["height"]]),  # 拖车右上角
}

In [5]:
# 遍历每个点，计算是否可见
for name, P  in trailer_corners.items():
    # 求镜像点和镜面的交点
    t_star = np.dot((M - P), n_Mirror) / np.dot((E_ref - P), n_Mirror)
    X = P + t_star * (E_ref - P)  # 交点坐标
    # 将交点转换到镜面局部坐标系
    local = X - M
    x_loc, y_loc = np.dot(local, u_h), np.dot(local, u_v)  # 在镜面局部坐标系下的坐标
    visible = (abs(x_loc) <= left_mirror["width"]/2) and (abs(y_loc) <= left_mirror["height"]/2) # 判断是否在镜面范围内
    print(name, "visible?" , visible, "| image coords =", (x_loc, y_loc))

left_bottom visible? True | image coords = (np.float64(-0.05720977412363109), np.float64(-0.0031197767342199226))
left_top visible? False | image coords = (np.float64(-0.05537863910500263), np.float64(-0.25680997558257107))
right_bottom visible? False | image coords = (np.float64(-0.29691292722017354), np.float64(-0.01619132493739391))
right_top visible? False | image coords = (np.float64(-0.2962328233326728), np.float64(-0.24314148840725197))
